https://deep-learning-study.tistory.com/376

https://github.com/dansuh17/alexnet-pytorch/blob/master/model.py

original : https://dl.acm.org/doi/pdf/10.1145/3065386


ImageNet Classification With Deep Convolutional Neural Networks

CNN 구조의 GPU 구현 및 dropout 적용



*dropout
- 통계 모델을 정규화하는 방법에 대해 가중치의 크기(L2 norm)을 패널티로 사용하여, 가중치 값을 강제로 축소.

딥 뉴럴 네트워크
- 각 특성을 독립적으로 보는 제약이 없기 때문에 편향 분산(bias-variance) 스펙트럼에서의 낮은 편향 및 높은 분산의 현상을 보임.
- 특성의 개수가 적은 경우에도 오버피팅될 가능성이 높음

--> 가중치 감쇠(weight decay) 및 l2 normalization을 통해 단항 함수를 사용해서 모델을 학습시키며 간단함을 유도

- 편향을 추가하지 않으면서 노이즈를 추가
: ϵ∼N(0,σ2)  노이즈를 입력에 더한 후  x′=x+ϵ  이 것을 학습 데이터로 사용

즉, 드롭아웃은 마지막 결과를 계산하기 위해 사용되는 연산의 몇몇 뉴런을 누락시킴.


사용한 데이터 : ImageNet dataset
- 1000 images with 1000 categories

#data preprocessing

1. 이미지 크기를 256*256
- resize방법은 이미지에서 짧은 쪽을 256으로 고정 후 중앙 부분을 256x256으로 crop
2. 각 이미지의 pixel에 training set의 평균 뺴기
- 각 이미지의 pixel에 training set의 평균을 빼서 normalize


#AlexNet Architecture

- AlexNet은 일부가 max-pooling layer가 적용된 5개의 convolutional layer와 3개의 fully-connected layer로 구성

- [Input layer- Conv1 - MaxPool1 - Norm1 - Conv2 - MaxPool2 - Norm2 - Conv3 - Conv4 - Conv5 - Maxpool3 - FC1 - FC2 -Output layer]

1. Input layer
: 227x227x3


2. Conv1
: 96 kernels, 11x11, stride=4, padding=0
- input : 224x224x3
- output : 55x55x96

3. MaxPool1
: 3x3 kernels, stride=2
- input : 55x55x96
- output : 27x27x96

4. Norm1
: LRN을 사용한 normalization layer
- input : 27x27x96
- output : 27x27x96

5. Conv2
: 256 kernels of size 5x5, stride=1, padding=2
- input : 27x27x96
- output : 27x27x256

6. MaxPool2
: 3x3 kernels, stride=2
- input : 27x27x96
- output : 13x13x256

7. Norm2
- input : 13x13x256
- output : 13x13x256

8. Conv3
: 384 kerenels of size 3x3, stride=1, padding=1
- input : 13x13x256
- output : 13x13x384

9. Conv4
: 384 kernels of size 3x3, stride=1, padding=1
- input : 13x13x384
- output : 13x13x384

10. Conv5
: 256 kernels fo size 3x3, stride=1, padding=1
- input : 13x13x384
- output : 13x13x256

11. MaxPool3
: 3x3 kernels, stride=2
- input : 13x13x256
- output : 6x6x256

12. FC1
: fc layer with 4096 neurons
- input : 6x6x256
- output : 4096

13. FC2
: fc layer with 4096 neurons
- input : 4096
- output : 4096

14. Output Layer
: fc layer with 1000-way softmax
- input : 4096
- output : 1000

AlexNet의 구조에 적용된 특징

1. ReLU Nonlinearity



2. Training on Multiple GPUs

: 2 GPU training (gpu parallelization)



데이터를 두 개의 gpu로 학습 후 하나의 layer에만 gpu를 통합.

3. Local Response Normalization(LRN)

: LRN은 generalization을 목적으로 한다. 본 논문은 ReLU를 사용하여 non-saturating nonlinearity의 특징으로 하여금 vanishing gradient를 제거.

또한 ReLU는 normalization이 필요하지 않음.

추가적으로 양수값을 받으면 그 값 그대로 neuron에 전달되기에 너무 큰 값이 전달되어 주변의 낮은 값이 neuron에 전달되는 것을 막을 수 있음.


AlexNet 이후 현대 CNN에는 LRN대신 batch normalization을 사용함

4. Overlapping Pooling

#Reducing Overfitting

1. Data Augmentation

: 적은 노력으로 다양한 데이터를 형성하여  overfitting 방지.

- generating image translation and horizontal reflections
: 256x256을 224x224로 crop(중앙, 좌우 상하) --> horizontal reflection

- altering the intensities of RGB channels in training images
: rgb pixel 값 변화

2. Dropout
: 매 입력마다 0.5의 확률로 순전파와 역전파를 진행

- AlexNet은 두 개의 FC 에서만 dropout 진행

#Details of learning
HyperParameter _ SGD(stochastic gradient descent)
- Momentum=0.9
- batch_size =128,
- weight_decay=0.005

vi+1 := 0.9*vi - 0.0005*e*wi - e*<əL/əW>

wi+1 := wi+vi+1



weight 초기화는 (0,0.01)
bias는 Conv2,4,5 Fc layer1, 나머지 layer는 0

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils from data

import torchvision.datasets import datasets
import torchvision.transforms as transforms

from tensorboardX import SummaryWriter

In [ ]:
#Pytorch devide 정의 / gpu사용
device = torch.device('cuda' if torch.cuda.is_availabel() else 'cpu')

In [ ]:
#model parameters 정의
NUM_EPOCHS = 90
BATCH_SIZE = 128
MOMENTUM = 0.9
LR_DECAY = 0.0005
LR_INIT = 0.01
IMAGE_DIM = 227 #pixels
NUM_CLASSES = 1000
DEVICE_IDS = [0,1,2,3]

In [ ]:
#data directory 지정
INPUT_ROOT_DIR = 'alexnet_data_in'
TRAIN_IMG_DIR = 'alexnet_data_in/imagenet'
OUTPUT_DIR = 'alexnet_data_out'
LOG_DIR = OUTPUT_DIR + '/tblogs'    #tensorboard logs
CHECKPOINT_DIR = OUTPUT_DIR + '/models'   #model checkpoints

#checkpoint 경로 directory 만들기
os.makedirs(CHECKPOINT_DIR, exist_ok = True)

In [ ]:
class AlexNet(nn.Module) :
  def __init__(self, num_classes=1000) :
    super().__init__()
    #input size : (bx3x227x227)
    #본 논문에서는 image 크기가 224pixel이라고 하지만, conv1d 이후 차원은 55x55
    #따라서 227x227로 변경
    self.net = nn.Sequential(
        nn.Conv2d(in_channel=3, out_channel=96, kernel_size=11, stride=4),
        nn.ReLU(),
        nn.LocalResponseNorm(size=5, alpha=0.001, beta = 0.75, k=2),#LRN    값은 논문에서 사용한 값을 사용
        nn.MaxPool2d(kernel=3, stride=2),     #maxpooling

        nn.Conv2d(96,256,5,padding=2),
        nn.ReLU(),
        nn.LocalResponseNorm(size=5, alpha=0.001, beta = 0.75, k=2),
        nn.MaxPool2d(kernel_size=3, stride=2),
        nn.ReLU(),

        nn.Conv2d(256,384,3, padding=1),
        nn.ReLU(),
        nn.Conv2d(384,384,3, padding=1),
        nn.ReLU(),
        nn.Conv2d(384,256,3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=3, stride=2)
    )

    #Fc Layer
    self.classifier = nn.Sequential(
        nn.Dropout(p=0.5, inplace=True),
        nn.Linear(in_features = (256*6*6), out_features = 4096),
        nn.ReLU(),

        nn.Dropout(p=0.5, inplace=True),
        nn.Linear(in_features = 4096, out_features = 4096),
        nn.LeRU(),

        nn.Linear(in_features = 4096, out_features = num_classes)
    )
    self.init_bias()


  def init_bias(self) :
    for layer in self.net :
      if isinstance(layer, nn.Conv2d) :
        #weight과 bias 초기화
        nn.init.nornam_(layer.weight, mean=0, std=0.1)
        nn.init.constant_(layer.bias, 0)
    #논문에서는 2,4,5 conv2d layer의 bias는 1로 초기화
    nn.init.constant_(self.net[4].bias, 1)
    nn.init.constant_(self.net[10].bias, 1)
    nn.init.constant_(self.net[12].bias, 1)


  def forward(self, x):
    x= self.net()
    x= x.view(-1,256*6*6)
  return self.classifier(x)


데이터 전처리, 손실함수, optimizer, 학습

In [ ]:
if __name__ == '__main__':
    # seed value 출력하기
    seed = torch.initial_seed()
    print('Used seed : {}'.format(seed))

    tbwriter = SummaryWriter(log_dir=LOG_DIR)
    print('TensorboardX summary writer created')

    # model 생성하기
    alexnet = AlexNet(num_classes=NUM_CLASSES).to(device)
    # 다수의 GPU에서 train
    alexnet = torch.nn.parallel.DataParallel(alexnet, device_ids=DEVICE_IDS)
    print(alexnet)
    print('AlexNet created')

    # dataset과 data loader 생성하기
    dataset = datasets.ImageFolder(TRAIN_IMG_DIR, transforms.Compose([
        # transforms.RandomResizedCrop(IMAGE_DIM, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
        transforms.CenterCrop(IMAGE_DIM),
        # transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]))
    print('Dataset created')
    dataloader = data.DataLoader(
        dataset,
        shuffle=True,
        pin_memory=True,
        num_workers=8,
        drop_last=True,
        batch_size=BATCH_SIZE)
    print('Dataloader created')

    # optimizer 생성하기
    optimizer = optim.SGD(
        params=alexnet.parameters(),      #alexnet의 parameter를 params에 저장
        lr=LR_INIT,
        momentum=MOMENTUM,      #momentum사용
        weight_decay=LR_DECAY)
    print('Optimizer created')

    # lr_scheduler로 LR 감소시키기 : 30epochs 마다 1/10
    lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
    print('LR Scheduler created')

    # train 시작
    print('Starting training...')
    total_steps = 1
    for epoch in range(NUM_EPOCHS):
        lr_scheduler.step()
        for imgs, classes in dataloader:
            imgs, classes = imgs.to(device), classes.to(device)

            # loss 계산
            output = alexnet(imgs)
            loss = F.cross_entropy(output, classes)

            # parameter 갱신
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # log the information and add to tensorboard
            # 정보를 기록하고 tensorboard에 추가하기
            if total_steps % 10 == 0:
                with torch.no_grad():
                    _, preds = torch.max(output, 1)
                    accuracy = torch.sum(preds == classes)

                    print('Epoch: {} \tStep: {} \tLoss: {:.4f} \tAcc: {}'
                        .format(epoch + 1, total_steps, loss.item(), accuracy.item()))
                    tbwriter.add_scalar('loss', loss.item(), total_steps)
                    tbwriter.add_scalar('accuracy', accuracy.item(), total_steps)

            # gradient values와 parameter average values 추력하기
            if total_steps % 100 == 0:
                with torch.no_grad():
                    # parameters의 grad 출력하고 저장하기
                    # parameters values 출력하고 저장하기
                    print('*' * 10)
                    for name, parameter in alexnet.named_parameters():
                        if parameter.grad is not None:
                            avg_grad = torch.mean(parameter.grad)
                            print('\t{} - grad_avg: {}'.format(name, avg_grad))
                            tbwriter.add_scalar('grad_avg/{}'.format(name), avg_grad.item(), total_steps)
                            tbwriter.add_histogram('grad/{}'.format(name),
                                    parameter.grad.cpu().numpy(), total_steps)
                        if parameter.data is not None:
                            avg_weight = torch.mean(parameter.data)
                            print('\t{} - param_avg: {}'.format(name, avg_weight))
                            tbwriter.add_histogram('weight/{}'.format(name),
                                    parameter.data.cpu().numpy(), total_steps)
                            tbwriter.add_scalar('weight_avg/{}'.format(name), avg_weight.item(), total_steps)

            total_steps += 1

        # checkpoints 저장하기
        checkpoint_path = os.path.join(CHECKPOINT_DIR, 'alexnet_states_e{}.pkl'.format(epoch + 1))
        state = {
            'epoch': epoch,
            'total_steps': total_steps,
            'optimizer': optimizer.state_dict(),
            'model': alexnet.state_dict(),
            'seed': seed,
        }
        torch.save(state, checkpoint_path)